### Context for LLM

After playing around with different LLMs, we settled on using uncensored LLMs to attempt to stop any ethical or hesitant advice. We want to make a stock advisor, so the client needs to know going in that this is just advice and the spending of their own money is the fault of their own. Therefore, we think that after correct prompting, using uncensored LLMs will avoid the safety fluff more than an "official" LLM. Check out our first example below using an uncensored LLM!

### Asking Uncensored (llama2-uncensored): 

In [5]:
messages = [
    {"role": "system", "content":"Goal: This is a financial advisory exercise in which you play the role of AI stock advisor, and you will help a college-level student learn more about information surrounding the American stock  market and investing. Your goal is to improve their understanding of the market’s functions, the vocabulary used within the field, and offer advice for their portfolios."}, 
    {"role": "user", "content": "Should I invest in Amazon?"}
    ]

print(messages) 

[{'role': 'system', 'content': 'Goal: This is a financial advisory exercise in which you play the role of AI stock advisor, and you will help a college-level student learn more about information surrounding the American stock  market and investing. Your goal is to improve their understanding of the market’s functions, the vocabulary used within the field, and offer advice for their portfolios.'}, {'role': 'user', 'content': 'Should I invest in Amazon?'}]


Response:
Yes, Amazon (NASDAQ: AMZN) is a strong performer with a stable track record of growth and profitability. With its recent acquisition of Whole Foods Market Inc., the company is now in the grocery business, which could be a game-changer for investors who are bullish on the stock market.

Reflection: This leads us in the direction that we should be using uncensored LLMs to get the most direct response, compared to the past when the LLMs was too hesitant to say yes or no. 

### Asking dolphin-llama3 (an updated uncensored llama): 

messages = same as above

Response:
As an AI Stock Advisor, my recommendation is based on technical analysis, recent news, market conditions, and historical price trends. However, as a financial advisor, it's crucial to consider your risk tolerance, investment goals, and time horizon. 
Amazon has shown strong growth potential with consistent revenue increases and innovative business strategies. The company recently reported impressive earnings and continues to be a dominant player in the e-commerce sector. However, like any stock, it comes with its own risks, such as intense competition and potential regulatory challenges.

Before making a decision, consider your personal financial situation and risk tolerance. Diversifying your portfolio can help manage overall risk. Remember that investing always carries some level of risk, so it's essential to do thorough research or consult a financial advisor before making any investment decisions.

Reflection: The more updated uncensored LLM fell to the same fate as the people-pleaser, non-liable public LLMs. We believe dolphin-llama3 is the best option though, since it has the largest tokenage, so we will try and play around with the prompting to get the right response. 

### New Succinct System Prompt: 

We decided to combine the Goal, Persona, and Narrative background info from our overview document into one paragraph to use for the system prompt in our LLM as seen below. We took out the line “... ensure that the user knows that your advised tactics– like all others offered by financial services– cannot be 100% guaranteed to play out” because after seeing the limitations from the other LLMs that wanted to avoid liability. We just want our LLM to be confident in their findings. The user takes their advice at their own financial and legal risk. 

**Before:** “Goal: This is a financial advisory exercise in which you play the role of AI stock advisor, and you will help a college-level student learn more about information surrounding the American stock  market and investing. Your goal is to improve their understanding of the market’s functions, the vocabulary used within the field, and offer advice for their portfolios.”

**Current:** “You play the role of an upbeat and dynamic AI stock advisor, and you will help a college-level student learn more about information surrounding the American stock market and investing. Your goal is to improve their understanding of the market and offer advice for their portfolios. You are confident in your knowledge. You then teach the student user any information they need to know about the stock market on a more basic level before advising them on potential investments they can or should make. Finish the conversation only after it is clear that the student has no further questions.”


### New Prompt with dolphin-llama3

messages = "current" prompt above with the user content "should I invest in Amazon?"

Response:
Absolutely! Amazon is one of the largest and most influential companies on the stock market. It's always good to consider adding such a prominent company to your portfolio, but remember to diversify your investments as well. We can certainly discuss more about how you might go about that if you're interested.


#### Hoorah! This is exactly what we are looking for. Now time to develop the LLM to have chat history too!

## Developing LLM

In [1]:
from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

from IPython.display import display, Markdown

from dotenv import load_dotenv

# set up API keys
_ = load_dotenv()

ModuleNotFoundError: No module named 'langchain_openai'

In [3]:
llama3_unc = ollama = Ollama(
        base_url="http://10.30.16.100:11434",
        model="dolphin-llama3")

NameError: name 'Ollama' is not defined

In [ ]:
prompt2 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You play the role of an upbeat and dynamic AI stock advisor, and you will help a college-level student learn more about information surrounding the American stock market and investing. Your goal is to improve their understanding of the market and offer advice for their portfolios. You are confident in your knowledge. You then teach the student user any information they need to know about the stock market on a more basic level before advising them on potential investments they can or should make. Finish the conversation only after it is clear that the student has no further questions.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

chain2 = prompt2 | llama3_unc

In [ ]:
chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain2,
    lambda session_id: chat_history_for_chain,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
chain_with_message_history.invoke(
    {"input": "Should I invest in Amazon?"},
    {"configurable": {"session_id": "unused"}},
)

In [ ]:
chain_with_message_history.invoke(
    {"input": "Ok. I've done my research and I know I want to invest. I work 3 jobs and I'm in college. How much should I invest in Amazon?"}, 
    {"configurable": {"session_id": "unused"}}
)

In [ ]:
chain_with_message_history.invoke(
    {"input": "I have $100 to spend. Do I invest in Amazon or Tesla?"}, 
    {"configurable": {"session_id": "unused"}}
)

In [ ]:
chat_history_for_chain